In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import datetime

import PIL.Image as Image
import tensorflow as tf
import tensorflow_hub as hub
%load_ext tensorboard


In [2]:
mobilenet_v2 ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
inception_v3 = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"

classifier_model = mobilenet_v2

In [5]:
data_root = tf.keras.utils.get_file(
  'flower_photos',
  'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

In [6]:
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [8]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

In [9]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [11]:
feature_extractor_layer = hub.KerasLayer(
    classifier_model,
    input_shape=(224, 224, 3),
    trainable=False)

In [12]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

(32, 1001)


In [13]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1001)              3540265   
_________________________________________________________________
dense (Dense)                (None, 5)                 5010      
Total params: 3,545,275
Trainable params: 5,010
Non-trainable params: 3,540,265
_________________________________________________________________


In [14]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1) # Enable histogram computation for every epoch.

In [15]:
NUM_EPOCHS = 10

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)

Epoch 1/10
92/92 [==============================] - 128s 1s/step - loss: 0.6969 - acc: 0.7548 - val_loss: 0.4421 - val_acc: 0.8542
Epoch 2/10
92/92 [==============================] - 111s 1s/step - loss: 0.3507 - acc: 0.8818 - val_loss: 0.3891 - val_acc: 0.8733
Epoch 3/10
92/92 [==============================] - 104s 1s/step - loss: 0.2773 - acc: 0.9022 - val_loss: 0.3743 - val_acc: 0.8760
Epoch 4/10
92/92 [==============================] - 96s 1s/step - loss: 0.2347 - acc: 0.9203 - val_loss: 0.3702 - val_acc: 0.8760
Epoch 5/10
92/92 [==============================] - 93s 1s/step - loss: 0.2056 - acc: 0.9312 - val_loss: 0.3704 - val_acc: 0.8801
Epoch 6/10
92/92 [==============================] - 96s 1s/step - loss: 0.1838 - acc: 0.9414 - val_loss: 0.3726 - val_acc: 0.8815
Epoch 7/10
92/92 [==============================] - 104s 1s/step - loss: 0.1664 - acc: 0.9506 - val_loss: 0.3759 - val_acc: 0.8815
Epoch 8/10
92/92 [==============================] - 107s 1s/step - loss: 0.1518 - acc: